#Autogluon解决灾难推文的自然语言处理
source : https://www.kaggle.com/competitions/nlp-getting-started/data?select=train.csv

In [1]:

!unzip nlp-getting-started.zip

Archive:  nlp-getting-started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
device = 'cuda'

In [4]:
data = pd.read_csv('train.csv').drop('id', axis=1)
test = pd.read_csv('test.csv')
test_id = test['id']
print(data.shape, test.shape,test_id.shape)

(7613, 4) (3263, 4) (3263,)


In [5]:
print('训练集\n',data.head())
print('测试集\n',test.head())

训练集
   keyword location                                               text  target
0     NaN      NaN  Our Deeds are the Reason of this #earthquake M...       1
1     NaN      NaN             Forest fire near La Ronge Sask. Canada       1
2     NaN      NaN  All residents asked to 'shelter in place' are ...       1
3     NaN      NaN  13,000 people receive #wildfires evacuation or...       1
4     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...       1
测试集
    id keyword location                                               text
0   0     NaN      NaN                 Just happened a terrible car crash
1   2     NaN      NaN  Heard about #earthquake is different cities, s...
2   3     NaN      NaN  there is a forest fire at spot pond, geese are...
3   9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
4  11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
#数据集拆分
data_train, data_eval = train_test_split(data, test_size=0.2, random_state=42,shuffle=True)
print(data_train.shape,data_eval.shape)

(6090, 4) (1523, 4)


In [7]:
#数据放入autogluon 专用dataframe
train_data = TabularDataset(data_train)
eval_data = TabularDataset(data_eval)
test_data = TabularDataset(test.drop('id', axis=1))

print(train_data.shape,eval_data.shape,test_data.shape)


(6090, 4) (1523, 4) (3263, 3)


In [8]:
#模型构建
predictor = TabularPredictor(label='target', eval_metric='f1',problem_type='binary',path='result')
model = predictor.fit(train_data, time_limit=1800, hyperparameters='multimodal',ag_args_fit={'num_gpus': 1})

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          8
Memory Avail:       49.02 GB / 50.99 GB (96.1%)
Disk Space Avail:   193.20 GB / 235.68 GB (82.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         : Strong accuracy with fast inference 

[1000]	valid_set's binary_logloss: 0.905941	valid_set's f1: 0.711019


	0.719	 = Validation score   (f1)
	17.12s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: MultiModalPredictor ... Training model for up to 1723.35s of the 1723.35s of remaining time.
INFO: Seed set to 0
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name              | Type                | Params | Mode 
------------------------------------------------------------------
0 | model             | MultimodalFusionMLP | 110 M  | train
1 | validation_metric | BinaryF1Score       | 0      | train
2 | loss_func         | CrossEntropyLoss    | 0      | train
------------------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
442.603   Total estimated model params size (MB)
305       Modules in train mode
0         Modules in eval mode
INFO: Epoch 0, global step 21: 'val_f1' reached 0.61180 (best 0.61180), saving model to '/content/result/models/MultiModalPredictor/automm_model/epoch=0-step=21.ckpt' as top 3
INFO: Epoch 0,

Predicting: |          | 0/? [00:00<?, ?it/s]

Calibrating decision threshold to optimize metric f1 | Checking 51 thresholds...
Calibrating decision threshold via fine-grained search | Checking 38 thresholds...
	Base Threshold: 0.500	| val: 0.7839
	Best Threshold: 0.343	| val: 0.7893
Updating predictor.decision_threshold from 0.5 -> 0.34299999999999997
	This will impact how prediction probabilities are converted to predictions in binary classification.
	Prediction probabilities of the positive class >0.34299999999999997 will be predicted as the positive class (1). This can significantly impact metric scores.
	You can update this value via `predictor.set_decision_threshold`.
	You can calculate an optimal decision threshold on the validation data via `predictor.calibrate_decision_threshold()`.
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/result")


In [9]:
y_pred = model.predict(eval_data)
y_pred

Load pretrained checkpoint: /content/result/models/MultiModalPredictor/automm_model/model.ckpt
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

,target
2644,0
2227,0
5448,1
132,0
6845,1
...,...
1835,1
506,1
3592,1
6740,1


In [10]:
#做出预测
eval_pred = model.evaluate(eval_data)
eval_pred

Load pretrained checkpoint: /content/result/models/MultiModalPredictor/automm_model/model.ckpt
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

{'f1': 0.7887527036770007,
 'accuracy': 0.8076165462902167,
 'balanced_accuracy': np.float64(0.8121498309315863),
 'mcc': np.float64(0.6177434584091016),
 'roc_auc': np.float64(0.8884598026183567),
 'precision': 0.7411924119241192,
 'recall': 0.8428351309707242}

In [11]:
model.leaderboard(eval_data)

Load pretrained checkpoint: /content/result/models/MultiModalPredictor/automm_model/model.ckpt
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,MultiModalPredictor,0.793328,0.780303,f1,4.984503,1.843902,671.978127,4.984503,1.843902,671.978127,1,True,7
1,WeightedEnsemble_L2,0.788753,0.783939,f1,5.186953,1.922711,724.729338,0.004039,0.002608,0.251452,2,True,8
2,LightGBMXT,0.749644,0.710204,f1,0.033280,0.012093,4.153161,0.033280,0.012093,4.153161,1,True,2
3,LightGBMLarge,0.737730,0.719008,f1,0.112859,0.026365,17.121634,0.112859,0.026365,17.121634,1,True,6
4,LightGBM,0.736240,0.713115,f1,0.030272,0.009214,12.069528,0.030272,0.009214,12.069528,1,True,1
5,XGBoost,0.724476,0.643172,f1,0.057500,0.020763,3.058630,0.057500,0.020763,3.058630,1,True,4
6,CatBoost,0.722261,0.695652,f1,0.050403,0.023302,15.959332,0.050403,0.023302,15.959332,1,True,3
7,NeuralNetTorch,0.655194,0.637275,f1,0.035149,0.026535,19.418793,0.035149,0.026535,19.418793,1,True,5


In [12]:
test_pred = model.predict(test_data)
test_pred

Load pretrained checkpoint: /content/result/models/MultiModalPredictor/automm_model/model.ckpt
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

,target
0,1
1,1
2,1
3,1
4,1
...,...
3258,1
3259,1
3260,1
3261,1


In [13]:
#提交
submission = pd.DataFrame({'id': test_id, 'target': test_pred})
submission.to_csv('submission.csv', index=False)